In [11]:
# Librerias
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import TransformadoresLocales as tfl


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Configuración de las visualizaciones
sns.set_theme(style='dark',palette='dark', font_scale=1.4)

# Cargar los datos
train_data_original = pd.read_csv('Data/train.csv')
test_data_original = pd.read_csv('Data/test.csv')

# Labels de cada columna obtenidos previamente por comodidad
numerical_keys = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_keys = ['PassengerId', 'HomePlanet', 'Cabin', 'Destination','CryoSleep', 'VIP', 'Name']

# Copia para hacer el análisis exploratorio
train_data = train_data_original.copy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocesamiento de los datos

Lo que haremos será diseñar un Pipeline con todos los cambios que queramos hacerle a los dato, este estudio fue realizado durante el análisis exploratorio de los datos.

En primer lugar tendremos el preprocesamiento de los datos, este consiste en preparar los datos para su posterior procesado. Primero se deben manejar los valores faltantes para luego aplicar transformadores en cada columna y asi obtener nuevos datos que sean mas eficientes para los siguientes pasos. El preprocesamiento nos permitirá obtener mejores predicciones, trabajando los datos originales y haciendo énfasis en sus partes mas significativas.

El siguiente paso será contruir uno o varios Pipelines que encadenen las transformaciones hechas durante el preprocesamiento y el entrenamiento de el modelo.

Se deben seleccionar múltiples modelos para ser sometidos a evaluacion cruzada y optimizar con métodos para buscar en el espacio de parámetros del pipeline completo.

Finalmente se debe realizar la evaluación final mediante con los datos conjunto independiente. Analizaremos estos errores en búsqueda de un mejor algoritmo.

## Primera estrategia

En el análisis vimos que de casi todas las columnas se pueden crear nuevas columnas obteniendo mejores datos. Primero entrenaremos diferentes modelos simples con cada uno de estos datos obtenidos para ver cual es el mas relevante.

Mas adelante combinaremos los datos procesados para entrenar otros tipos de modelos mas complejos como redes neuronales.


In [12]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [13]:
## agarro todos los transformadores independientes para cada columna y los junto en un puro transformador
transformador_columnas = ColumnTransformer(
    transformers=[
        ('passenger_id_trans',  tfl.PassengerId_Transformer(),  ['PassengerId']),
        ('itinerario_trans',    tfl.Itinerario_Transformer(),   ['HomePlanet', 'Destination']),
        ('cabin_trans',         tfl.Cabin_Transformer(),        ['Cabin']),
        ('age_trans',           tfl.Age_Transformer(),          ['Age']),
        ('name_trans',          tfl.Name_Transformer(),         ['Name']),
        ('gastos_trans',        tfl.Gastos_Transformer(),       ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck','CryoSleep', 'Age'])
    ]
)

# El pipeline final que se ejecuta es el que cunta el preprocesamiento con el entrenamiento de los modelos
preprocesador = Pipeline(steps=[
    ('imputador',       tfl.Imputer_Transformer()),
    ('preprocesador',   transformador_columnas),
    ('poner labels',    tfl.VolverADataFrame_Transformer() )
])


In [14]:
nuevo = preprocesador.fit_transform(X = train_data, y=train_data['Transported'])
nuevo['Transported'] = train_data['Transported']
nuevo.head()


,GrupoId,GrupoSize,Solo,T-T,A_trappist,terricola,Piso,N_cabina,Lado,Zona,Rango etario,MenorDeEdad,NFAbordo,Gasto Total,GastoMayor,Transported
0,1,1,1,False,True,False,B,0,P,Zona 1,Adulto,False,1,0.0,True,False
1,2,1,1,True,True,True,F,0,S,Zona 1,Adulto Joven,False,4,736.0,False,True
2,3,2,0,False,True,False,A,0,S,Zona 1,Adulto Mayor,False,6,10383.0,False,False
3,3,2,0,False,True,False,A,0,S,Zona 1,Adulto,False,6,5176.0,False,False
4,4,1,1,True,True,True,F,1,S,Zona 1,Menor,True,6,1091.0,False,True


In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# 1. Crear transformadores como funciones ajustables
def select_columns(X, columns=None):
    return X[columns] if columns else X

# Transformador para selección de columnas
column_selector = FunctionTransformer(select_columns, kw_args={'columns': None})

# 2. Crear pipelines independientes para cada grupo
feature_groups = {
    'gastos': ['Gasto Total', 'GastoMayor'],
    'grupo': ['GrupoSize', 'Solo', 'NFAbordo'],
    #'ubicacion': ['Piso', 'Lado', 'Zona'],
    #'demograficas': ['Rango etario', 'MenorDeEdad'],
    #'itinerario': ['T-T', 'A_trappist', 'terricola']
}

models = {
    'logreg': LogisticRegression(max_iter=1000, random_state=42, ),
    'tree': DecisionTreeClassifier(random_state=42),
    'knn': KNeighborsClassifier(),
    'svm': SVC(probability=True, random_state=42)
}

# Pipelines para cada grupo de características
group_pipelines = {}
for group_name, columns in feature_groups.items():
    group_pipelines[group_name] = make_pipeline(
        FunctionTransformer(select_columns, kw_args={'columns': columns}),
        # Aquí puedes agregar más transformaciones específicas para el grupo
    )

# 3. Pipeline principal ajustado
preprocesador = Pipeline(steps=[
    ('imputador', tfl.Imputer_Transformer()),
    ('preprocesador', transformador_columnas),
    ('poner_labels', tfl.VolverADataFrame_Transformer())
])

# Ajustar el preprocesador con los datos COMPLETOS
X_full = train_data.drop('Transported', axis=1)
y_full = train_data['Transported']
preprocesador.fit(X_full, y_full)  # ¡Ajustar aquí primero!

# 4. Función de evaluación corregida
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

for group_name, group_pipe in group_pipelines.items():
    print(f"\nEvaluando grupo: {group_name}")
    
    # Transformar los datos una vez con el preprocesador ajustado
    X_transformed = preprocesador.transform(X_full)
    
    for model_name, model in models.items():
        # Crear pipeline completo para este grupo/modelo
        full_pipe = make_pipeline(
            clone(group_pipe),  # Clonar el pipeline del grupo
            model
        )
        
        scores = cross_val_score(full_pipe, X_transformed, y_full, 
                                cv=5, scoring='accuracy')
        print(f"{model_name}: {scores.mean():.4f} (±{scores.std():.4f})")




Evaluando grupo: gastos


c:\Users\dorla\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


logreg: 0.7233 (±0.0128)
tree: 0.6941 (±0.0096)
knn: 0.6590 (±0.0902)
